In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
from mapping import ResourceMapper
from graph import *
from sparql import SPAQLConverter

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')
rm = ResourceMapper(config)
cgg = ConceptualGraphGenerator(config)
qgg = QueryGraphGenerator(config)
sc = SPAQLConverter(config)

# 2-1. Query Partitioning

In [29]:
# user_input = '가수 태연의 첫 데뷔곡은?'
# user_input = '뱅뱅뱅 부른 음악그룹의 소속사는?'
# user_input = '용감한 형제가 작곡한 노래의 장르는?'
# user_input = '장르가 발라드인 노래'
# user_input = '버벌진트가 피처링한 노래'
# user_input = '소녀시대의 리더는?'
user_input = '제니의 소속 그룹'
# user_input = '레인을 작곡한 사람'
query_terms = rm.tokenize_query(user_input)
query_terms

['제니', '소속', '그룹']

In [30]:
query_patitions = rm.partition_query(query_terms)
query_patitions

[['제니 소속 그룹'], ['제니', '소속 그룹'], ['제니 소속', '그룹'], ['제니', '소속', '그룹']]

# 2-2. Partition Scoring

In [31]:
p_star, score_list = rm.score_partition(query_patitions)
p_star

['제니', '소속', '그룹']

# 2-3. Generation of resource combination

In [32]:
resource_combinations = rm.combinate_resource(p_star)
resource_combinations

[('skmo:제니', 'mo:member_of', 'foaf:Group'),
 ('skmo:제니', 'schema:affiliation', 'foaf:Group')]

# 3. Conceptual graph generation

In [33]:
conceptual_graph = cgg.generate_conceptual_graph(resource_combinations)
conceptual_graph

[[('skmo:제니', 'mo:member_of', 'foaf:Group')],
 [('skmo:제니', 'schema:affiliation', 'foaf:Group')]]

# 4. Query graph generation

In [34]:
query_graph = qgg.generate_query_graph(conceptual_graph)
query_graph

[(0.5,
  [('foaf:Person(skmo:제니)', 'mo:member_of', 'mo:MusicGroup'),
   ('mo:MusicGroup', 'rdfs:subClassOf', 'foaf:Group')])]

# 5. SPARQL Conversion

In [35]:
final_query_graph = sc.select_query_graph(query_graph)
final_query_graph

[('foaf:Person(skmo:제니)', 'mo:member_of', 'mo:MusicGroup'),
 ('mo:MusicGroup', 'rdfs:subClassOf', 'foaf:Group')]

In [36]:
relation_triples = sc.extract_relation_triples(final_query_graph)
relation_triples

[('foaf:Person(skmo:제니)', 'mo:member_of', 'mo:MusicGroup')]

In [37]:
knows_query = sc.generate_query(relation_triples)
print(knows_query)

SELECT DISTINCT ?target 
WHERE
{
    skmo:제니 mo:member_of ?target
}


In [38]:
result = sc.excute_query(knows_query)
for r in result:
    print(r['target'])

http://www.sktelecom.com/skmo/블랙핑크
